In [84]:
# Importing metadata zip file and converting it to dataframe

import json
import gzip
import pandas as pd

def getDF(path):
  # Unzip the file, load in each line as an object
  g = gzip.open(path, 'rb')
  g = [json.loads(l) for l in g]

  # Map to a dictionary, then load in as a dataframe
  dict_df = {i: d for (i, d) in enumerate(g)}
  return pd.DataFrame.from_dict(dict_df, orient='index')

df = getDF('meta_ALL_Beauty.json.gz')

In [85]:

# Selecting only the colums that are required for analysis

colums_description_asin = ["description","asin"]
df = df[colums_description_asin]

df.describe()

,description,asin
count,32892,32892
unique,13751,32488
top,[],B00027CDOW
freq,17773,2


In [86]:
# Descriptions in the json file are a list of strings, converting to sting for easy cleaning
df["newdescription"] = df.description.map(lambda x: ".".join(x).replace("\n",""))


In [87]:
# Remove empty descriptions
#df = df.drop(df[df["newdescription"] == ""].index)

#print(len(df)) 



In [88]:
df = df.drop(df[ ( (df["newdescription"] == "" ) | ( df["newdescription"].isnull()) ) ].index)

df

,description,asin,newdescription
0,[Loud 'N Clear Personal Sound Amplifier allows...,6546546450,Loud 'N Clear Personal Sound Amplifier allows ...
1,[No7 Lift & Luminate Triple Action Serum 50ml ...,7178680776,No7 Lift & Luminate Triple Action Serum 50ml b...
2,[No7 Stay Perfect Foundation now stays perfect...,7250468162,No7 Stay Perfect Foundation now stays perfect ...
4,[Lacto Calamine Skin Balance Daily Nourishing ...,7414204790,Lacto Calamine Skin Balance Daily Nourishing L...
5,[Mary Kay Satin Hands Peach Hand Cream Travel ...,7535842801,Mary Kay Satin Hands Peach Hand Cream Travel S...
...,...,...,...
32880,[Move over soap on a rope! This heavy-duty Bri...,B01HIHLFOC,Move over soap on a rope! This heavy-duty Bric...
32884,[Eau de parfum spray vial mini design house: y...,B01HIPOQ2M,Eau de parfum spray vial mini design house: yv...
32885,[Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stu...,B01HIUEEHO,Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stuf...
32886,[New and unused product. 100% authentic Benefi...,B01HIWKGOM,New and unused product. 100% authentic Benefit...


In [89]:
# Remove duplicate asin so can have only unique products
df[df.duplicated("asin")]
df = df.drop_duplicates("asin", keep="last")
df.describe()

,description,asin,newdescription
count,14821,14821,14821
unique,13749,14821,13743
top,"[For over 60 years, Betty Dain Creations, Inc....",6546546450,"For over 60 years, Betty Dain Creations, Inc. ..."
freq,59,1,59


In [90]:
# Remove duplicate description
df[df.duplicated("newdescription")]
df = df.drop_duplicates("newdescription", keep="last")
df.describe() 

,description,asin,newdescription
count,13743,13743,13743
unique,13743,13743,13743
top,[Loud 'N Clear Personal Sound Amplifier allows...,6546546450,Loud 'N Clear Personal Sound Amplifier allows ...
freq,1,1,1


In [91]:
# Removing redundant old description column
colums_description_asin = ["newdescription","asin"]
df = df[colums_description_asin]

df.describe()

,newdescription,asin
count,13743,13743
unique,13743,13743
top,Loud 'N Clear Personal Sound Amplifier allows ...,6546546450
freq,1,1


In [92]:
# Identify range of descriptions in after duplicates removal

df_descriptions_without_empty = df["newdescription"].apply(lambda x: len(x.split()))

df_descriptions_without_empty.describe([0.1,0.15,0.20,0.25,0.30,0.75,0.85,0.90,0.92,0.95,0.97,0.98,0.99])

# 97% upper limit of 259 (same % milit as reviews)
# 11 words (same numbe of words as reviews) 20% because no assuption is made on how much more each one is informative as well as descriptions 
# being a smaller sample


count    13743.000000
mean        63.643600
std         91.940065
min          1.000000
10%          7.000000
15%          9.000000
20%         11.000000
25%         14.000000
30%         17.000000
50%         39.000000
75%         79.000000
85%        114.000000
90%        148.000000
92%        169.000000
95%        217.000000
97%        259.000000
98%        288.000000
99%        324.000000
max       3224.000000
Name: newdescription, dtype: float64

In [93]:
# Remove descriptions with more or less than xx pre-cleaned words

# Split at any white space 
df["num_words_description"] = df["newdescription"].apply(lambda x: len(x.split()))

# Check if under or equal to 80% upper limmit words fulfils withs condition and set it
df = df[(df["num_words_description"] <= 259) & (df["num_words_description"] >= 11)]

df["newdescription"].describe()

count                                                 10710
unique                                                10710
top       Loud 'N Clear Personal Sound Amplifier allows ...
freq                                                      1
Name: newdescription, dtype: object

In [94]:
# Check shortening worked
df_descriptions_without_empty = df["newdescription"].apply(lambda x: len(x.split()))

df_descriptions_without_empty.describe([0.1,0.15,0.20,0.25,0.30,0.75,0.85,0.90,0.92,0.95,0.97,0.98,0.99])


count    10710.000000
mean        65.241923
std         53.053433
min         11.000000
10%         15.000000
15%         18.000000
20%         23.000000
25%         27.000000
30%         31.000000
50%         49.000000
75%         86.000000
85%        115.000000
90%        142.000000
92%        157.000000
95%        185.000000
97%        211.000000
98%        225.000000
99%        243.000000
max        259.000000
Name: newdescription, dtype: float64

In [95]:
# Regrex for character removal
import re

# Spacy for spell check
import spacy
import contextualSpellCheck
nlp = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(nlp)

# NLTK for tokenisation and lemmatization
import nltk

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')

# Spacy stop word creation
#stopping_words = spacy.lang.en.stop_words.STOP_WORDS
#stopping_words_new = stopping_words

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
stopping_words_new = list(text.ENGLISH_STOP_WORDS)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
stopping_words_to_remove = ['without' , 'not', 'less', 'noting', 'none','no']
for word in list(stopping_words_new):
    if word in stopping_words_to_remove:
        stopping_words_new.remove(word)
print(stopping_words_new)

['many', 'whatever', 'those', 'through', 'while', 'if', 'then', 'least', 'ltd', 'see', 'when', 'name', 'should', 'cry', 'everywhere', 'a', 'was', 'noone', 'go', 'next', 'almost', 'namely', 'this', 'one', 'top', 'though', 'only', 'already', 'amount', 'somehow', 'keep', 'hence', 'herself', 'any', 'fifty', 'thus', 'give', 'ever', 'can', 'down', 'toward', 'with', 'why', 'everyone', 'couldnt', 'last', 'onto', 'becoming', 'anyone', 'myself', 'hers', 'besides', 'hereupon', 'detail', 'someone', 'thereupon', 'your', 'nobody', 'what', 'too', 'something', 'into', 'nothing', 'thru', 'whom', 'therefore', 'eight', 'nevertheless', 'never', 'side', 'against', 'had', 'ten', 'yourself', 'back', 'be', 'were', 'again', 'bill', 'itself', 'nowhere', 'being', 'seeming', 'fifteen', 'because', 'which', 'perhaps', 'seemed', 'for', 'its', 'from', 'therein', 'etc', 'ours', 'until', 'five', 'neither', 'co', 'his', 'anything', 'forty', 'yourselves', 'bottom', 'indeed', 'becomes', 'here', 'below', 'himself', 'behind

In [97]:
# Preprocessing of descriptions

def preprocessing(raw_string):
    # Remove html tags and anything inside them 
    no_html = re.sub(r'<[^>]*>','', raw_string)
    #print("after removing html", no_html)


    # Make everything lowercase
    lowercase_column = no_html.lower()
    #print("lowercase", lowercase_column)

    # Remove apostrophe to enable spell check to correct words with apostrophe
    without_apostrophe = re.sub(r'[\']', '', lowercase_column)

    # ! Need to double check again where best to use this spell check
    # 
    # .pipe for batches of text
    #doc = list(nlp.pipe(without_apostrophe))
    #doc = nlp(without_apostrophe)

    #spell_checked = doc._.outcome_spellCheck

    # Remove all non alphabetic instances that aren't a space and replace them with a space using Regrex
    alphabetic_column = re.sub(r'[^a-z\s]', ' ', without_apostrophe)
    #print("removed numerical and punctuation", alphabetic_column)

    # Tokenize string into individual words
    tokens = word_tokenize(alphabetic_column)

    # Remove stopping words using Spacy library
    tokens_without_stopping_words = [token for token in tokens if token not in stopping_words_new]

    # Lemmatize tokens using nltk and join them into sentances
    sentances_without_stop_words = ' '.join([lemmatizer.lemmatize(t) for t in tokens_without_stopping_words])

    return sentances_without_stop_words


In [98]:
df["clean_description"] = df["newdescription"].head(10000).apply(preprocessing)

df["clean_description"].to_csv("out_new.csv")

df["clean_description"]

0        loud n clear personal sound amplifier allows t...
2        no stay perfect foundation stay perfect longer...
4        lacto calamine skin balance daily nourishing l...
5        mary kay satin hand peach hand cream travel si...
7        according legend brother native origin black b...
                               ...                        
32879                                                  NaN
32880                                                  NaN
32884                                                  NaN
32885                                                  NaN
32890                                                  NaN
Name: clean_description, Length: 10710, dtype: object

In [99]:
# Remove any empty descriptions that appear because of head()
df = df.drop(df[ ( (df["clean_description"] == "" ) | ( df["clean_description"].isnull()) ) ].index)

In [100]:
# Identifying range of descriptions in after cleaning
 
df_description_without_empty_clean = df["clean_description"].apply(lambda x: len(x.split()))
#df_descriptions_without_empty.describe()
df_description_without_empty_clean.describe([0.03,0.1,0.2,0.3,0.75,0.85,0.90,0.95])

count    10000.000000
mean        42.043000
std         32.748768
min          2.000000
3%           9.000000
10%         11.000000
20%         16.000000
30%         21.000000
50%         32.000000
75%         55.000000
85%         73.000000
90%         88.000000
95%        113.000000
max        241.000000
Name: clean_description, dtype: float64

In [101]:
from collections import Counter
Counter(" ".join(df["clean_description"]).split()).most_common(100)

[('skin', 6307),
 ('hair', 3894),
 ('oil', 3222),
 ('use', 2500),
 ('product', 2420),
 ('color', 2356),
 ('natural', 2107),
 ('body', 1905),
 ('help', 1738),
 ('not', 1640),
 ('x', 1639),
 ('free', 1558),
 ('oz', 1491),
 ('size', 1452),
 ('quality', 1450),
 ('fragrance', 1427),
 ('no', 1389),
 ('dry', 1280),
 ('high', 1224),
 ('make', 1193),
 ('soft', 1137),
 ('ingredient', 1132),
 ('perfect', 1106),
 ('new', 1106),
 ('formula', 1088),
 ('day', 1080),
 ('clean', 1049),
 ('water', 1005),
 ('smooth', 998),
 ('used', 962),
 ('eye', 960),
 ('look', 951),
 ('cm', 951),
 ('long', 949),
 ('cream', 916),
 ('easy', 914),
 ('great', 904),
 ('vitamin', 859),
 ('lip', 858),
 ('feature', 850),
 ('nail', 842),
 ('time', 841),
 ('blend', 837),
 ('face', 824),
 ('contains', 815),
 ('material', 814),
 ('scent', 807),
 ('black', 791),
 ('essential', 791),
 ('extract', 788),
 ('set', 778),
 ('brush', 773),
 ('design', 772),
 ('hand', 760),
 ('soap', 753),
 ('style', 751),
 ('care', 742),
 ('brand', 740),

In [108]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
print(list(text.ENGLISH_STOP_WORDS))

['many', 'whatever', 'those', 'through', 'while', 'if', 'then', 'without', 'least', 'ltd', 'see', 'when', 'name', 'should', 'cry', 'everywhere', 'a', 'was', 'noone', 'go', 'next', 'almost', 'namely', 'this', 'one', 'top', 'though', 'only', 'already', 'amount', 'somehow', 'keep', 'hence', 'herself', 'any', 'fifty', 'thus', 'give', 'ever', 'can', 'down', 'toward', 'with', 'why', 'everyone', 'couldnt', 'last', 'onto', 'less', 'becoming', 'anyone', 'myself', 'hers', 'besides', 'hereupon', 'detail', 'someone', 'thereupon', 'your', 'nobody', 'what', 'too', 'something', 'into', 'nothing', 'thru', 'whom', 'therefore', 'eight', 'nevertheless', 'never', 'side', 'against', 'had', 'ten', 'yourself', 'back', 'be', 'were', 'again', 'bill', 'itself', 'nowhere', 'being', 'seeming', 'fifteen', 'because', 'which', 'perhaps', 'seemed', 'for', 'its', 'from', 'therein', 'etc', 'ours', 'until', 'five', 'neither', 'co', 'his', 'anything', 'forty', 'yourselves', 'bottom', 'indeed', 'becomes', 'here', 'below',

In [103]:
###############################################################
#                      Cleaning Keywords                                

# TODO use finalise list of keywords

enviromental = ['biodegradable', 'reduced packaging', 'reduced', 'sustainable', 'plastic-free', 'sustainably sourced', 'compostable', 'renewable', 'renewable energy', 'reusable', 'biodegradable', 'organic', 'refillable', 'refills', 'solid bar', 'recycled', 'cardboard', 'reef safe','reef-friendly', 'oxybenzone free', 'triclosan-free', 'microplastics free', 'microbeads free', 'palm oil free', 'HDPE', 'post-consumer recycled plastic', 'renewable energy', 'recycling scheme', 'sustainably sourced', 'low-impact', 'carbon neutral', 'carbon offsetting', 'eco', 'soil association', 'conservation', 'COSMOS', 'NATRUE', 'RSPO', 'FSC']
social = ['No animal testing', 'cruelty-free', 'vegan', 'plant-based', 'palm oil-free', 'ethical', 'vegan society', 'PETA', 'leaping bunny', 'fair trade', 'local', 'hand-made', 'small business'] 
economic = ['Fair trade', 'renewable energy', 'circular economy', 'locally sourced', 'local', 'small business', 'job creation']
health = ['non toxic', 'bio', 'organic', 'plant based', 'paraben free', 'triclosan free', 'fragrance free', 'synthetic fragrance free', 'SLS free', 'phthalates free','nanoparticles free', 'non nano', 'formaldehyde free', 'phthalates free', 'no GMO', 'soil association', 'COSMOS', 'NATRUE', 'USDA']
idnk = ['palm oil-free','free','no gmo','free from','no plastic','no chemicals']

# Cleaning keywords in the same way as the corpus, 
# splitting them into individual words (for vectorisation) 
# removing duplicates, 
# converts it to a list
enviro_processed = list(set(word for phrase in enviromental for word in preprocessing(phrase).split()))
social_processed = list(set(word for phrase in social for word in preprocessing(phrase).split()))
economic_processed = list(set(word for phrase in economic for word in preprocessing(phrase).split()))
health_processed = list(set(word for phrase in health for word in preprocessing(phrase).split()))

#enviro_processed = set(map(preprocessing, enviromental))
print(enviro_processed)
print(social_processed)
print(economic_processed)
print(health_processed)



['biodegradable', 'organic', 'cosmos', 'triclosan', 'plastic', 'post', 'eco', 'sustainably', 'microplastics', 'microbeads', 'recycling', 'soil', 'consumer', 'refillable', 'rspo', 'free', 'packaging', 'low', 'safe', 'scheme', 'compostable', 'energy', 'sourced', 'conservation', 'reef', 'palm', 'hdpe', 'oil', 'sustainable', 'reduced', 'oxybenzone', 'neutral', 'reusable', 'friendly', 'bar', 'impact', 'fsc', 'offsetting', 'association', 'solid', 'natrue', 'recycled', 'carbon', 'cardboard', 'refill', 'renewable']
['no', 'leaping', 'business', 'fair', 'small', 'bunny', 'trade', 'based', 'plant', 'local', 'free', 'animal', 'society', 'peta', 'palm', 'oil', 'hand', 'vegan', 'ethical', 'testing', 'cruelty']
['trade', 'economy', 'business', 'fair', 'energy', 'local', 'small', 'sourced', 'job', 'creation', 'circular', 'locally', 'renewable']
['no', 'organic', 'paraben', 'cosmos', 'nanoparticles', 'triclosan', 'soil', 'toxic', 'based', 'plant', 'free', 'synthetic', 'phthalates', 'usda', 'bio', 'nan

In [104]:
#                      Testing negative keywords

non_enviromental = ['packaging','plastic',  'oxybenzone', 'triclosan', 'microplastics', 'microbeads', 'palm oil']
non_social = ['animal testing', 'palm oil','imported'] 
non_health = ['toxic', 'paraben', 'triclosan','SLS', 'phthalates','nanoparticles', 'nano', 'formaldehyde', 'phthalates', 'GMO']

# Cleaning keywords in the same way as the corpus
non_enviromental_processed = list(set(word for phrase in non_enviromental for word in preprocessing(phrase).split()))
non_social_processed = list(set(word for phrase in non_social for word in preprocessing(phrase).split()))
non_health_processed = list(set(word for phrase in non_health for word in preprocessing(phrase).split()))


print(non_enviromental_processed)
print(non_social_processed)
print(non_health_processed)


['microplastics', 'microbeads', 'plastic', 'oil', 'palm', 'triclosan', 'packaging', 'oxybenzone']
['imported', 'animal', 'testing', 'palm', 'oil']
['sl', 'nano', 'toxic', 'formaldehyde', 'gmo', 'paraben', 'nanoparticles', 'triclosan', 'phthalates']


In [143]:
###############################################################  SAVEEEE THISSS
#                      CorEx                                  #
# Code based on
# https://github.com/gregversteeg/corex_topic/blob/mastera/corextopic/example/corex_topic_example.ipynb

# Setting anchor words for corEx
# Anchor with group of words
anchor_words = [enviro_processed, social_processed, economic_processed, health_processed]#, non_enviromental_processed, non_social_processed,non_health_processed]#, non_clean_enviro, non_clean_social, non_clean_health]

#------------------------------------------------------
#                   Vectorisation
# fow working with sparse arrays
import scipy.sparse as ss

# Vectorisation
from sklearn.feature_extraction.text import CountVectorizer

# Vectorise the dataset

# DONT NEED for description but might need for reviews check if max_features=20000 is needed, if there is a point to use it as it will select top words from everything, number might need adjusting
# DONT NEED for decription but might need fore reviews invest9gate the chance of using Max_df to remove some words with high frequency

vectorizer = CountVectorizer(binary=True)         
doc_word = vectorizer.fit_transform(df["clean_description"]) # dont need .data cuz we have dataframe
# Transform descriptions into a sparse matrix
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape) # n_docs x m_words

# Getting words for labeling the columns of the matrix
corpus_words = vectorizer.get_feature_names_out()

combined_words = list(corpus_words) + list(enviro_processed) + list(social_processed) + list(economic_processed) + list(health_processed)

combined_words = list(set(combined_words))
vectorizer = CountVectorizer(vocabulary=combined_words, binary=True)

doc_word = vectorizer.transform(df["clean_description"])
doc_word = ss.csr_matrix(doc_word)
print(doc_word.shape)
#-----------------------------------------------------
#           COREX
import corextopic.corextopic as ct

# TODO experiment with anchor strength
anchored_topic_model = ct.Corex(n_hidden=4, seed = 2)
anchored_topic_model.fit(doc_word, words=combined_words, anchors=anchor_words, anchor_strength=5);

for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ', '.join(topic_words))



(10000, 26030)
(10000, 26037)
0: use, safe, product, free, not, oil, organic, used, result, using
1: skin, oil, ingredient, vitamin, free, natural, help, formula, extract, moisture
2: material, package, feature, easy, cm, quality, high, size, included, fit
3: fragrance, organic, scent, blend, free, plant, essential, musk, note, lavender
-----------------
0: safe, oil, organic, free, friendly, use, product, used, energy, low
1: oil, skin, free, ingredient, vitamin, natural, plant, formula, help, extract
2: small, not, business, easy, feature, time, fit, make, need, energy
3: fragrance, organic, plant, free, paraben, scent, toxic, based, non, blend
-----------------
0: oil, free, organic, safe, friendly, low, energy, palm, use, product
1: oil, skin, free, plant, ingredient, vitamin, natural, extract, animal, help
2: small, not, business, energy, easy, job, time, using, make, feature
3: fragrance, organic, free, plant, paraben, non, no, based, toxic, scent
-----------------
0: oil, organi

In [ ]:
'''0: oil, free, organic, safe, friendly, low, energy, palm, post, eco
1: oil, free, skin, plant, animal, based, vegan, ingredient, no, natural
2: small, energy, business, not, use, fair, product, time, job, circular
3: free, no, non, organic, based, plant, synthetic, paraben, toxic, fragrance
With new stopwords
0: oil, free, organic, safe, low, friendly, biodegradable, energy, natural, reusable
1: oil, free, hand, animal, based, small, vegan, testing, cruelty, palm
2: small, energy, fair, warranty, feature, circular, designed, cut, width, control
3: free, non, organic, based, plant, paraben, synthetic, toxic, association, gmo
4: oil, plastic, packaging, palm, seed, almond, jojoba, peppermint, triclosan, coconut
5: oil, animal, testing, essential, skin, ingredient, pure, vitamin, tea, butter
6: paraben, blade, gmo, shave, trimmer, norelco, razor, cordless, norelcos, shaving
Test
0: oil, organic, safe, plastic, low, friendly, packaging, refill, palm, free
1: oil, free, hand, animal, based, testing, vegan, cruelty, palm, local
2: small, fair, trimmer, head, cordless, blade, shaver, norelcos, shave, norelco
3: fragrance, free, organic, non, based, synthetic, paraben, toxic, gmo, association
4: oil, packaging, palm, result, tea, mineral, tree, property, coconut, triclosan
5: oil, animal, testing, palm, essential, natural, ingredient, vitamin, calendula, consult
6: paraben, skin, gmo, help, healthy, aging, extract, product, acid, tone



With stop words ( based on 2 it looks like some of the words lile LOCAL, LOSS and CIRCULAR are excluded unfortunately)

0: organic, safe, oil, friendly, free, refill, energy, palm, biodegradable, reusable
1: oil, free, animal, plant, vegan, cruelty, testing, palm, hand, based
2: small, local, fair, trimmer, cordless, contour, norelcos, norelco, worldwide, warranty
3: fragrance, free, organic, non, based, plant, synthetic, paraben, toxic, gmo
'''

'''
After splitting words

0: oil, free, organic, bar, packaging, friendly, biodegradable, palm, sustainable, soap
1: oil, animal, no, hand, based, free, vegan, testing, cruelty, small
2: small, energy, fair, local, circular, loss, economy, creation, trade, aging
3: fragrance, free, organic, non, based, synthetic, paraben, toxic, blend, gmo
'''

'''
Before splitting words

0: organic, refill, biodegradable, reusable, fragrance, recommended, launched, house, note, sustainable
1: hand, vegan, local, trimmer, cordless, contour, norelcos, charging, norelco, warranty
2: organic, ingredient, extract, vitamin, skin, bio, natural, product, oil, rich
3: free, cruelty, non, radical, sensitive, gluten, contact, sunscreen, paraben, apply
'''

'''
Withouth spell check

0: organic, fragrance, biodegradable, launched, house, note, scent, recommended, blend, refill
1: hand, vegan, trimmer, cordless, head, shaving, blade, shave, system, norelco
2: local, diet, aging, similarly, consider, instance, whenapplyingany, alter, oilyskin, chemistry
3: organic, vitamin, oil, ingredient, skin, extract, natural, animal, product, healthy
'''

In [ ]:
# Show top words for selected topic, number is "highest mutual information with the topic"
anchored_topic_model.get_topics(topic=2, n_words=50)

In [ ]:
# Show all documents for each topic
# TODO what is exactly this number?
anchored_topic_model.get_top_docs(topic=1, n_docs=10000, sort_by='log_prob')#[-20:]

In [ ]:
import corextopic.vis_topic as vt
vt.vis_rep(anchored_topic_model, column_label=words, prefix='twenty')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Read labels from labels text file in twenty
data = np.loadtxt('twenty\labels.txt', delimiter=',')

# Extract all columns but not the first one
columns = data[:, 1:]

# Summation of each column to add up how often each theme appears
sum_themes = np.sum(columns, axis=0)

# TODO if non of the rows = 1 then make label 5 (no theme) =+

# Identify and sum any rows without 1s in each column by checking if the sum of the whole row is 0
rows_without_theme = np.sum(columns, axis=1) == 0

# Sum
num_rows_without_theme = np.sum(rows_without_theme)
print(num_rows_without_theme)

# Add new column represting no themes classified with 1 as no theme and 0 having a theme
data = np.column_stack((data, rows_without_theme.astype(int)))

# Create new labels text file with addition of the new "no themes" column as int
np.savetxt( 'new_lables.txt', data, delimiter=',', fmt='%d')


# Display the results
labels = ['Enviromental', 'Social', 'Economic', 'Health']
plt.bar(labels, sum_themes)
plt.xlabel('Themes')
plt.ylabel('N of times')
plt.title('Number of times each theme appears')
plt.show()

In [158]:
# from https://www.kaggle.com/code/nvpsani/topic-modelling-using-guided-lda 
# used workaround as well

import pandas as pd
import numpy as np
from lda import guidedlda as glda

from sklearn.feature_extraction.text import CountVectorizer

# Anchor words list
anchor_words = [enviro_processed, social_processed, economic_processed, health_processed]

# Create corpus of descriptions
descriptions_corpus = df["clean_description"].tolist() 

# Create vocabulary to work with 
vocab = list(set(word_tokenize(" ".join(df["clean_description"]))))
print(vocab)

vocab.extend(enviro_processed)
vocab.extend(social_processed)
vocab.extend(economic_processed)
vocab.extend(health_processed)
vocab = list(set(vocab))
print(vocab)
print("vocab",len(vocab))

vectorizer = CountVectorizer(ngram_range = (1,1), vocabulary = vocab)
vectorised_descriptions_corpus = vectorizer.fit_transform(descriptions_corpus)
word2id = vectorizer.vocabulary_

# Defining model
model = glda.GuidedLDA(n_topics=4, n_iter=200, random_state=6, refresh=20,alpha=0.01,eta=0.01) 

# Topics for the model
anchor_topics = {}
for t_id, st in enumerate(anchor_words):
    for word in st:
        anchor_topics[word2id[word]] = t_id
        
#train model
model.fit(vectorised_descriptions_corpus, seed_topics=anchor_topics, seed_confidence=0.90)

['vienne', 'novelette', 'dishwasher', 'schweibildung', 'kay', 'mobil', 'sho', 'aromat', 'rivet', 'sleek', 'p', 'relieffor', 'ecology', 'toddler', 'misty', 'giza', 'handpiece', 'altera', 'cross', 'exfoliatingeffectively', 'tvlink', 'locking', 'goer', 'packthe', 'pounding', 'bronzage', 'velvetcolor', 'illustration', 'charleston', 'lograr', 'barn', 'customizing', 'picturesweight', 'fishing', 'costume', 'neighbor', 'amoena', 'brighteners', 'sambac', 'immunity', 'keratin', 'jam', 'waver', 'religion', 'amor', 'breakdown', 'cluttered', 'skyblue', 'radient', 'popcorn', 'scintillates', 'mmpost', 'runaway', 'polysilicone', 'wheat', 'rebalances', 'entices', 'colorwont', 'fish', 'motorfor', 'strategically', 'curb', 'hull', 'occasionally', 'whand', 'cleomee', 'savage', 'acupoint', 'spellbound', 'eisenhower', 'vous', 'reddish', 'powderthis', 'devotion', 'novelty', 'periodontitis', 'dense', 'metome', 'glycyrrhiza', 'drippers', 'hc', 'dingrdients', 'downside', 'obedience', 'aa', 'nozzlemade', 'skineve

INFO:lda:n_documents: 10000
INFO:lda:vocab_size: 26061
INFO:lda:n_words: 414505
INFO:lda:n_topics: 4
INFO:lda:n_iter: 200
INFO:lda:<0> log likelihood: -4263354
INFO:lda:<20> log likelihood: -3524999
INFO:lda:<40> log likelihood: -3465134
INFO:lda:<60> log likelihood: -3438525
INFO:lda:<80> log likelihood: -3429486
INFO:lda:<100> log likelihood: -3425319
INFO:lda:<120> log likelihood: -3422163
INFO:lda:<140> log likelihood: -3419873
INFO:lda:<160> log likelihood: -3419169
INFO:lda:<180> log likelihood: -3418056
INFO:lda:<199> log likelihood: -3418239


In [159]:
top_n_glda_words = 10
words_in_topics = model.topic_word_
for i, topic_dist in enumerate(words_in_topics):
    words_in_this_topic = np.array(vocab)[np.argsort(topic_dist)][:-(top_n_glda_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(words_in_this_topic)))

Topic 0: use hair nail not product easy size brush clean feature
Topic 1: skin hair oil natural help use free formula ingredient body
Topic 2: color cm quality size high hair material new brand product
Topic 3: fragrance oil oz scent body soap product note blend essential


In [ ]:

'''
conifence 0.85



New Stopwords
Topic 0: skin oil hair natural help use formula free product body
Topic 1: hair product not color easy size use designed bra brush
Topic 2: fragrance scent soap body note blend bath oz natural woman
Topic 3: shave blade head shaving razor xl trimmer shaver hair norelco

TEST
Topic 0: skin fragrance hair natural body help oil free vitamin dry blend formula oz smooth scent clean use extract note nbsp cream fresh leaf rich soft wear moisture color gentle healthy amp sensitive gently face contains treatment lotion organic line acid woman recommended anti protection looking shine butter apply scalp essential
Topic 1: shave blade shaving head razor trimmer shaver hair clean minute fit bra norelco comfortable skin cut day use time cup come easy brush year cartridge guarantee contour smooth cordless teeth close battery shape feature hour comfort warranty charge norelcos beard face designed comb cord bristle provides adjust facial length charging
Topic 2: oil product soap use bath ingredient natural formula quality body oz relief water pure pheromone organic bottle bar essential foot hand day help child healing lasting tree hot soothing long available gel health skin professional effect spray effective way gift area liquid contains care time item bubble flavor result cold
Topic 3: hair product color xl lip easy size soft lash perfect use great high line water set designed brush long feature look cover oz pack i

WITH STOP WORDS
Topic 0: fragrance scent soap note xl blend oz wear body design woman recommended smell house
Topic 1: hair shave blade shaving head razor trimmer shaver easy minute feature comfortable brush norelco
Topic 2: skin oil natural hair help body free use product formula clean vitamin oz ingredient
Topic 3: product use bra size lash cup great relief easy color shape woman available designed

Wihtout stop words normal
Topic 0: shave hair blade head shaving razor system trimmer xl easy shaver brush use comfortable
Topic 1: skin oil hair natural help free formula use body vitamin clean oz dry smooth
Topic 2: product not use bath body natural water relief foot child no cold professional tree
Topic 3: fragrance scent soap note woman blend body wear oz bra design recommended house smell

?
Topic 0: no hand safe small non free plastic biodegradable fragrance reusable low toxic palm based
Topic 1: oil no hand organic animal fragrance bar plant small refill testing packaging based friendly
Topic 2: fragrance bar based low energy oil bio packaging carbon creation local business economy sustainably
Topic 3: free oil organic non fragrance no animal vegan safe cruelty hand friendly paraben synthetic

OLD VECTORISATION WITH KEYWORDS

Topic 0: shave hair blade head shaving razor xl trimmer system easy
Topic 1: skin hair oil natural body help free use formula vitamin
Topic 2: product not use bath oil natural relief water child ingredient
Topic 3: fragrance scent soap note blend oz product woman wear design

NEW VECTORISATION WITH KEYWORDS

Topic 0: fragrance scent soap body note blend oz woman bath wear
Topic 1: skin oil hair natural help use formula free product body
Topic 2: hair product size easy color designed day bra brush soft
Topic 3: shave blade shaving head razor xl trimmer shaver hair norelco

OLD VECTORISATION WITHOUT KEYWORDS

pic 0: skin hair help natural free formula oil body vitamin dry
Topic 1: fragrance scent note product woman body blend wear oz design
Topic 2: hair shave shaving blade head razor trimmer easy shaver fit
Topic 3: oil soap product organic use bath natural ingredient xl no


'''